In [2]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [3]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_file = 'data/train-balanced-sarcasm.csv'
data = pd.read_csv(data_file)
data.dropna(subset=['comment'], inplace=True)
data.head()
data = pd.DataFrame({"text":data['comment'], "label":data['label']})

train, test = train_test_split(data, test_size=0.1)
train, valid = train_test_split(train, test_size=0.1)

In [46]:
train.shape

(818725, 2)

In [47]:
valid.shape

(90970, 2)

In [48]:
test.shape

(101078, 2)

In [53]:
MAX_VOCAB_SIZE = 25_000
vector = 'data/glove.840B.300d'

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

100%|█████████▉| 399665/400000 [00:20<00:00, 18928.45it/s]

In [68]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = BATCH_SIZE, 
    device = device)

AttributeError: 'DataFrame' object has no attribute 'BucketIterator'

In [54]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [55]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 901 trainable parameters


In [57]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.1172e-01, -4.9659e-01,  1.6307e-01, -8.8169e-01,  5.3900e-02,
          6.6837e-01, -5.9658e-02, -4.6750e-01, -2.1525e-01,  8.8396e-01,
         -7.5842e-01, -3.6887e-01, -3.4239e-01, -1.4020e+00,  3.2065e-01,
         -1.0219e+00,  7.9883e-01, -9.2288e-02, -7.0492e-01, -1.6024e+00,
          2.8910e-01,  4.8987e-01, -3.8527e-01, -7.1203e-01, -1.7061e-01,
         -1.4594e+00,  2.2066e-01,  2.4625e-01, -1.3248e+00,  6.9698e-01,
         -6.6305e-01,  1.2158e+00, -1.4949e+00,  8.8096e-01, -1.1786e+00,
         -9.3400e-01, -5.6755e-01, -2.7723e-01, -2.1834e+00,  3.6683e-01,
          9.3803e-01,  7.8165e-03, -3.1386e-01, -1.1567e+00,  1.8409e+00,
         -1.0174e+00,  1.2192e+00,  1.6013e-01,  1.5985e+00, -4.6861e-02,
         -1.5270e+00, -2.0143e+00, -1.5173e+00,  3.8774e-01, -1.1849e+00,
          6.8967e-01,  1.3232e+00,  1.8169e+00,  6.8077e-01,  7.2437e-01,
          3.2311e-02, -1.6593e+00, -1.8773e+00,  7.3725e-01,  9.2574e-01,
          9.2470e-01,  1.8254e-01, -7.

In [58]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [59]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [62]:
criterion = nn.BCEWithLogitsLoss()

model = model.to('cpu')
criterion = criterion.to('cpu')

In [63]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [64]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [66]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [67]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

NameError: name 'train_iterator' is not defined

In [ ]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

100%|█████████▉| 399665/400000 [00:39<00:00, 18928.45it/s]